# Tarea 3: Piping y Bombas
### Pablo Correa e Ian Gross

## Librerías

In [5]:
import CoolProp.CoolProp as cp
import fluids as fld
import numpy as np
import scipy.constants as cte
import scipy.optimize as opt
import matplotlib.pyplot as plt
import pandas as pd

from fluids.units import *

In [21]:
g = cte.g*u.m/u.s**2
Patm = (1*u.atm).to(u.Pa)

# Pregunta 1
Considere el sistema de almacenamiento de energía visto en el laboratorio.

A partir de los datos de operación del archivo anexo, verifique si la pérdida de carga observada se puede representar por la ecuación de Ergun.

¿Cuánto es el error con respecto a la correlación? (3pts), ¿Cuanto afecta la temperatura y la variabilidad de las propiedades termofísicas del aire? (3pts).

Asuma que el sistema de almacenamiento es un intercambiador de calor de flujo paralelo, donde el solido es uno de los medio de transferencia de calor. ¿Cuanto sería la efectividad del sistema (intercambiador de calor)? (3pts)

In [17]:
headers = ["Scan Sweep Time (s)", "Temp. Entrada (°C)", "Vel. Aire (m/s)", "Temp. Aire (°C)","Presión Dif. Estanque (mbar)", "Temp. Salida (°C)"]
data = pd.read_excel("Ensayo Piloto Lleno T-220°C.xlsx", skiprows=16, usecols="A,C:G", names=headers)
data

,Scan Sweep Time (s),Temp. Entrada (°C),Vel. Aire (m/s),Temp. Aire (°C),Presión Dif. Estanque (mbar),Temp. Salida (°C)
0,2024-09-25 09:49:13.522,14.079405,6.472987,13.120901,0.992468,17.538146
1,2024-09-25 09:49:23.515,14.078369,6.656212,13.121559,1.165138,17.564700
2,2024-09-25 09:49:33.515,14.084538,6.437116,13.121187,1.213865,17.573132
3,2024-09-25 09:49:43.515,14.115000,6.546101,13.121459,1.051219,17.570347
4,2024-09-25 09:49:53.515,14.115085,6.437106,13.121394,1.138136,17.589255
...,...,...,...,...,...,...
2193,2024-09-25 15:54:43.516,30.401201,6.624077,18.110073,-0.993292,19.313422
2194,2024-09-25 15:54:53.516,30.379024,6.654350,17.866438,-0.811574,19.329506
2195,2024-09-25 15:55:03.516,30.373109,6.654364,18.111046,-0.891405,19.376573
2196,2024-09-25 15:55:13.516,30.396446,6.623994,18.111007,-1.021911,19.408551


In [19]:
data.corr()

,Temp. Entrada (°C),Vel. Aire (m/s),Temp. Aire (°C),Presión Dif. Estanque (mbar),Temp. Salida (°C)
Temp. Entrada (°C),1.000000,0.586682,0.779613,0.665450,0.745914
Vel. Aire (m/s),0.586682,1.000000,0.815704,0.820089,0.520827
Temp. Aire (°C),0.779613,0.815704,1.000000,0.963905,0.798176
Presión Dif. Estanque (mbar),0.665450,0.820089,0.963905,1.000000,0.736789
Temp. Salida (°C),0.745914,0.520827,0.798176,0.736789,1.000000


# Pregunta 2
Liquid  carbon  dioxide  enters  the  tubes  of  a  shell  and  tube  heat  exchanger  at  a  temperature of 0°C and a flow rate of 30 kg/s. The carbon dioxide is heated with a flow of city water that enters the shell side of the heat exchanger at a temperature of 40°C and a flow rate of 31 kg/s. The heat exchanger is a single shell, two-tube pass configuration with a 25in shell. The tubes are ¾in 10BWG laid out on a 1in triangular pitch. The heat exchanger is 2m long and contains 2 baffles per meter of length.
* Asumiendo presión atmosférica en la entrada

In [34]:
T_CO2 = (0*u.degC).to(u.K)
dm_CO2 = 30*u.kg/u.s
T_CityWater = (40*u.degC).to(u.K)
dm_CityWater = 31*u.kg/u.s

d_shell = (25*u.inch).to(u.m)
do_tube = (0.75*u.inch).to(u.m)
t_tube = t_from_gauge(10, schedule='BWG')
di_tube = do_tube - t_tube
l = 2*u.m
A_shell = np.pi * d_shell * l
A_tube = np.pi * di_tube * l

# Fouling Factor
R_CO2 = 0.000176*u.m**2*u.K/u.W
R_CityWater = 0.000352*u.m**2*u.K/u.W


In [43]:
P_CO2 = cp.PropsSI('P', 'T', T_CO2.magnitude, 'Q', 0, 'CO2')*u.Pa

cp_CO2 = cp.PropsSI('Cpmass', 'T', T_CO2.magnitude, 'Q', 0, 'CO2')*u.J/u.kg/u.K
cp_CityWater = cp.PropsSI('Cpmass', 'T', T_CityWater.magnitude, 'P', Patm.magnitude, 'water')*u.J/u.kg/u.K
Cmin = min(dm_CO2*cp_CO2, dm_CityWater*cp_CityWater)
Cmax = max(dm_CO2*cp_CO2, dm_CityWater*cp_CityWater)
Cr = Cmin/Cmax

rho_CO2 = cp.PropsSI('D', 'T', T_CO2.magnitude, 'Q', 0, 'CO2')*u.kg/u.m**3
mu_CO2 = cp.PropsSI('V', 'T', T_CO2.magnitude, 'Q', 0, 'CO2')*u.Pa*u.s

rho_CityWater = cp.PropsSI('D', 'T', T_CityWater.magnitude, 'P', Patm.magnitude, 'water')*u.kg/u.m**3
mu_CityWater = cp.PropsSI('V', 'T', T_CityWater.magnitude, 'P', Patm.magnitude, 'water')*u.Pa*u.s
k_CityWater = cp.PropsSI('L', 'T', T_CityWater.magnitude, 'P', Patm.magnitude, 'water')*u.W/u.m/u.K

V_CO2 = dm_CO2/(rho_CO2*A_tube)
V_CityWater = dm_CityWater/(rho_CityWater*A_shell)

Re_CityWater = fld.core.Reynolds(V_CityWater, d_shell, rho_CityWater, mu_CityWater).to(u.dimensionless)
Pr_CityWater = fld.core.Prandtl(Cp=cp_CityWater, mu=mu_CityWater, k=k_CityWater).to(u.dimensionless)
print(Re_CityWater, Pr_CityWater)

7558.734639617106 dimensionless 4.340630370366401 dimensionless


In [ ]:
Qmax = Cmin*(T_CityWater-T_CO2)

# Single shell double tube heat exchanger
e = lambda NTU, Cr: 2*(1+Cr+np.sqrt(1+Cr**2)*(1+np.exp(-NTU*np.sqrt(1+Cr**2)))/(1-np.exp(-NTU*np.sqrt(1+Cr**2))))**-1


a. The outlet temperatures and pressure drops of each fluid in the heat exchanger for the case of a brand new heat exchanger just put into service.

In [ ]:
print(Re_CO2.to(u.dimensionless))
print

47555.105742957145 dimensionless


b. The outlet temperatures of each fluid after the heat exchanger has been in continuous service for one year without maintenancece.

c. If the outlet temperature of the carbon dioxide must be 20°C or higher, is this heat exchanger sufficient for the required duty?

# Pregunta 3
Considere el plano hidráulico de la localidad de Hualañé y la distribución de diámetros determinada en la Tarea 2. Seleccione una bomba de uno de los dos catalogos disponibles, que permita entregar el caudal y nivel de presión requerido (6pts). Identifique el punto de operación de la bomba (6pts)

# Bibliografía